In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unidecode import unidecode
import plotly.graph_objects as go

%matplotlib inline

# Candidatos a vereadores pelo PSOL

In [7]:
df = pd.read_csv('input/tse/vereadores_psol/2020.csv').drop(['Unnamed: 0'],axis=1)

# Regiões e capitais

In [17]:
sigla_estado = {'AC':'Acre',
                'AL':'Alagoas',
                'AP':'Amapa',
                'AM':'Amazonas',
                'BA':'Bahia',
                'CE':'Ceara',
                'DF':'Distrito Federal',
                'ES':'Espirito Santo',
                'GO':'Goias',
                'MA':'Maranhao',
                'MT':'Mato Grosso',
                'MS':'Mato Grosso do Sul',
                'MG':'Minas Gerais',
                'PA':'Para',
                'PB':'Paraiba',
                'PR':'Parana',
                'PE':'Pernambuco',
                'PI':'Piaui',
                'RJ':'Rio de Janeiro',
                'RN':'Rio Grande do Norte',
                'RS':'Rio Grande do Sul',
                'RO':'Rondonia',
                'RR':'Roraima',
                'SC':'Santa Catarina',
                'SP':'Sao Paulo',
                'SE':'Sergipe',
                'TO':'Tocantins'}

In [9]:
capitais = {'AC':'Rio Branco',
            'AL':'Maceio',
            'AP':'Macapa',
            'AM':'Manaus',
            'BA':'Salvador',
            'CE':'Fortaleza',
            'DF':'Brasilia',
            'ES':'Vitoria',
            'GO':'Goiania',
            'MA':'Sao Luis',
            'MT':'Cuiaba',
            'MS':'Campo Grande',
            'MG':'Belo Horizonte',
            'PA':'Belem',
            'PB':'Joao Pessoa',
            'PR':'Curitiba',
            'PE':'Recife',
            'PI':'Teresina',
            'RJ':'Rio de Janeiro',
            'RN':'Natal',
            'RS':'Porto Alegre',
            'RO':'Porto Velho',
            'RR':'Boa Vista',
            'SC':'Florianopolis',
            'SP':'Sao Paulo',
            'SE':'Aracaju',
            'TO':'Palmas'}
capitais = dict((k.upper(), v.upper()) for k, v in capitais.items())

In [10]:
df['capitais'] = pd.Series(list(zip(df.sigla_unidade_federativa, df.nome_municipio))).isin(capitais.items())

In [11]:
regioes = {'Norte':['AC','AP','AM','PA','RO','RR','TO'],
           'Nordeste':['AL','BA','CE','MA','PB','PE','PI','RN','SE'],
           'Centro-Oeste':['GO','MT','MS'],
           'Sudeste':['ES','MG','RJ','SP'],
           'Sul':['PR','SC','RS']}

In [12]:
df['regiao'] = df.sigla_unidade_federativa.apply(lambda x: ''.join([k for k, v in regioes.items() if x in v]))

# Lista de população, tamanho e densidade por município

In [18]:
url = 'https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_do_Brasil_por_popula%C3%A7%C3%A3o_(2020)'
municipio_pop = pd.read_html(url, thousands=' ')[0]

# eliminar espaço para converter para int
municipio_pop['População'] = municipio_pop['População'].str.replace('\D', '').astype(int)

# converter estado e municipio para sem acentos e maiusculo
municipio_pop['Unidade federativa'] = municipio_pop['Unidade federativa'].str.upper().apply(lambda x: unidecode(x))
municipio_pop['Município'] = municipio_pop['Município'].str.upper().apply(lambda x: unidecode(x))

# abreviar UF
municipio_pop['sigla_uf'] = municipio_pop['Unidade federativa'].str.upper().apply(lambda x: [k for k, v in sigla_estado.items() if (unidecode(v.upper()) == x)])
municipio_pop = municipio_pop.explode('sigla_uf')

municipio_pop.drop('Posição', axis=1, inplace=True)
municipio_pop.set_index('Código IBGE', inplace=True)

In [19]:
url = 'https://pt.m.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_brasileiros_por_%C3%A1rea_decrescente'
municipio_km = pd.read_html(url, thousands=' ')[0]

municipio_km['Área (km²)'] = municipio_km['Área (km²)'].str.replace(',','.').str.replace(u'\xa0', u'').astype(float)
municipio_km.drop(['Posição','Município', 'Unidade federativa'], axis=1, inplace=True)
municipio_km.rename(columns={'Código do IBGE':'Código IBGE'}, inplace=True)
municipio_km.set_index('Código IBGE', inplace=True)

In [20]:
density_label = ['0-5','5-10','10-20','20-50','50-100','100-200','200-20k']

def agrupar_por_densidade(den):
    if den >= 0 and den < 5:
        return 0
    elif den >= 5 and den < 10:
        return 1
    elif den >= 10 and den < 20:
        return 2
    elif den >= 20 and den < 50:
        return 3
    elif den >= 50 and den < 100:
        return 4
    elif den >= 100 and den < 200:
        return 5
    elif den >= 200 and den < 20000:
        return 6
    else:
        return 'other'
    
municipios = pd.merge(municipio_pop, municipio_km, on='Código IBGE')
municipios['densidade'] = municipios['População'] / municipios['Área (km²)']
municipios['grupo_densidade'] = municipios.apply(lambda x : agrupar_por_densidade(x.densidade), axis=1)

In [21]:
# Obter tabela de correspondencia entre codigo de municipio do IBGE e TSE
link = 'https://raw.githubusercontent.com/estadao/como-votou-sua-vizinhanca/master/data/votos/correspondencia-tse-ibge.csv'
df_equivalencia = pd.read_csv(link)
df_equivalencia.set_index('COD_TSE', drop=False, inplace=True)
df_equivalencia.drop(['chave','NOME','UF'], axis=1, inplace=True)

In [22]:
df.set_index('codigo_municipio', drop=False, inplace=True)
df = df.join(df_equivalencia)

In [23]:
df.set_index('GEOCOD_IBGE', drop=False, inplace=True)
df = df.join(municipios[['População','densidade','grupo_densidade']])

In [24]:
# Obter tabela latitude e longitude para municipios
link = 'https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/municipios.csv'
df_lat_long = pd.read_csv(link)
df_lat_long.set_index('codigo_ibge', inplace=True)
df_lat_long.drop(['nome','capital','codigo_uf'], axis=1, inplace=True)

In [25]:
df = df.join(df_lat_long)

# Candidatos por tamanho do município

In [38]:
GRANDE = 500000
PEQUENO = 100000

Total de candidatos 

In [36]:
df.shape[0]

3337

In [65]:
df[(df.totalizacao_turno.isin(['ELEITO','ELEITO POR MEDIA','ELEITO POR QP']))].shape[0]

82

In [75]:
82/3337

0.024572969733293377

Total de candidatos em capitais

In [66]:
df[df.capitais].shape[0]

433

In [67]:
df[(df.capitais) & (df.totalizacao_turno.isin(['ELEITO','ELEITO POR MEDIA','ELEITO POR QP']))].shape[0]

30

In [76]:
30/433

0.06928406466512702

Total de candidatos em cidades grandes

In [68]:
df[df['População'] >= GRANDE].shape[0]

690

In [69]:
df[(df['População'] >= GRANDE) & (df.totalizacao_turno.isin(['ELEITO','ELEITO POR MEDIA','ELEITO POR QP']))].shape[0]

40

In [77]:
40/690

0.057971014492753624

Total de candidatos em cidades medias

In [70]:
df[(df['População'] >= PEQUENO) & (df['População'] < GRANDE)].shape[0]

1056

In [71]:
df[(df['População'] >= PEQUENO) & (df['População'] < GRANDE) & (df.totalizacao_turno.isin(['ELEITO','ELEITO POR MEDIA','ELEITO POR QP']))].shape[0]

9

In [78]:
9/1056

0.008522727272727272

Total de candidatos em cidades pequenas

In [72]:
df[df['População'] < PEQUENO].shape[0]

1591

In [73]:
df[(df['População'] < PEQUENO) & (df.totalizacao_turno.isin(['ELEITO','ELEITO POR MEDIA','ELEITO POR QP']))].shape[0]

33

In [79]:
33/1591

0.020741671904462602

In [115]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = ["Candidatos", "Capitais", "Outras Grandes Grandes", "Cidades Médias", "Cidades Pequenas",
               "Eleitos"],
      color = "blue"
    ),
    link = dict(
      source = [0, 0, 0, 0, 1, 2, 3, 4],
      target = [1, 2, 3, 4, 5, 5, 5, 5],
      value = [433, 257, 1056, 1591, 30, 10, 9, 33]
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()